In [1]:
!pip install mediapipe

In [2]:
import cv2
import mediapipe as mp
import numpy as np
import math
from IPython.display import display
import ipywidgets as widgets

# Specify the path to your uploaded video in Colab.
video_path = "/content/WhatsApp Video 2025-02-22 at 13.40.45_bce8d8b3.mp4"  # Replace with your actual video file name

# Initialize MediaPipe Holistic (which includes face mesh landmarks).
mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic(
    static_image_mode=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)
mp_draw = mp.solutions.drawing_utils

# Create an ipywidgets Image widget for fast, in-place frame updates.
img_widget = widgets.Image(format='jpeg')
display(img_widget)

def rotationMatrixToEulerAngles(R):
    """
    Convert a rotation matrix R to Euler angles (in radians).
    The returned angles are in the order [pitch, yaw, roll].
    """
    sy = math.sqrt(R[0, 0] ** 2 + R[1, 0] ** 2)
    singular = sy < 1e-6
    if not singular:
        x = math.atan2(R[2, 1], R[2, 2])
        y = math.atan2(-R[2, 0], sy)
        z = math.atan2(R[1, 0], R[0, 0])
    else:
        x = math.atan2(-R[1, 2], R[1, 1])
        y = math.atan2(-R[2, 0], sy)
        z = 0
    return np.array([x, y, z])

cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Finished processing video.")
        break

    # Convert frame from BGR to RGB for processing.
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = holistic.process(frame_rgb)

    # If face landmarks are detected:
    if results.face_landmarks:
        # Draw the full face mesh using face_mesh connections.
        mp_draw.draw_landmarks(frame, results.face_landmarks, mp.solutions.face_mesh.FACEMESH_TESSELATION)
        h, w, _ = frame.shape
        face_landmarks = results.face_landmarks.landmark

        # Use a subset of 6 face landmarks for head pose estimation:
        # Indices: Nose tip (1), Chin (152), Left eye left corner (33),
        # Right eye right corner (263), Left mouth corner (61), Right mouth corner (291)
        if len(face_landmarks) >= 468:  # Ensure the full set is available.
            image_points = np.array([
                (face_landmarks[1].x * w,   face_landmarks[1].y * h),    # Nose tip
                (face_landmarks[152].x * w, face_landmarks[152].y * h),    # Chin
                (face_landmarks[33].x * w,  face_landmarks[33].y * h),     # Left eye left corner
                (face_landmarks[263].x * w, face_landmarks[263].y * h),    # Right eye right corner
                (face_landmarks[61].x * w,  face_landmarks[61].y * h),     # Left mouth corner
                (face_landmarks[291].x * w, face_landmarks[291].y * h)     # Right mouth corner
            ], dtype=np.float64)

            # 3D model points of a generic face model (in millimeters).
            model_points = np.array([
                [0.0,    0.0,    0.0],      # Nose tip
                [0.0,   -63.6,  -12.5],      # Chin
                [-43.3,  32.7,  -26.0],      # Left eye left corner
                [43.3,   32.7,  -26.0],      # Right eye right corner
                [-28.9, -28.9,  -24.1],      # Left mouth corner
                [28.9,  -28.9,  -24.1]       # Right mouth corner
            ], dtype=np.float64)

            # Camera internals (assuming no lens distortion).
            focal_length = w
            center = (w / 2, h / 2)
            camera_matrix = np.array([
                [focal_length, 0, center[0]],
                [0, focal_length, center[1]],
                [0, 0, 1]
            ], dtype="double")
            dist_coeffs = np.zeros((4, 1))  # No lens distortion.

            # Solve for head pose using solvePnP.
            success, rotation_vector, translation_vector = cv2.solvePnP(
                model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_ITERATIVE
            )
            if success:
                # Project a 3D point (0, 0, 1000.0) onto the image plane to visualize head direction.
                (nose_end_point2D, _) = cv2.projectPoints(
                    np.array([(0.0, 0.0, 1000.0)]), rotation_vector, translation_vector, camera_matrix, dist_coeffs
                )
                p1 = (int(image_points[0][0]), int(image_points[0][1]))  # Nose tip.
                p2 = (int(nose_end_point2D[0][0][0]), int(nose_end_point2D[0][0][1]))
                cv2.line(frame, p1, p2, (255, 0, 0), 2)

                # Convert rotation vector to Euler angles.
                rmat, _ = cv2.Rodrigues(rotation_vector)
                euler_angles = rotationMatrixToEulerAngles(rmat)
                euler_angles_deg = np.degrees(euler_angles)
                pitch, yaw, roll = euler_angles_deg

                # Flag "cheating" if yaw exceeds a threshold (i.e. head turned too far).
                threshold = 15  # degrees
                if abs(yaw) > threshold:
                    cv2.putText(frame, f"Face Cheating! Yaw: {yaw:.2f}", (30, 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                else:
                    cv2.putText(frame, f"Face Normal. Yaw: {yaw:.2f}", (30, 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                # Display the 3D coordinates from the translation vector.
                nose_3d = translation_vector.flatten()  # in millimeters.
                coord_text = f"3D: X:{nose_3d[0]:.1f} Y:{nose_3d[1]:.1f} Z:{nose_3d[2]:.1f}"
                cv2.putText(frame, coord_text, (30, 60),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

                # Display the normalized MediaPipe face landmark (nose tip) coordinates.
                mp_point = face_landmarks[1]
                mp_point_text = f"MP: x:{mp_point.x:.3f} y:{mp_point.y:.3f} z:{mp_point.z:.3f}"
                cv2.putText(frame, mp_point_text, (30, 90),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

    # Encode the processed frame as a JPEG image.
    ret2, buffer = cv2.imencode('.jpg', frame)
    if not ret2:
        continue
    # Update the ipywidgets Image widget for fast, in-place display.
    img_widget.value = buffer.tobytes()

cap.release()
print("Processing complete.")


ImportError: DLL load failed while importing _framework_bindings: A dynamic link library (DLL) initialization routine failed.

In [5]:
import cv2
import mediapipe as mp
import numpy as np
import math
from IPython.display import display

# Specify your video file (ensure it's uploaded to Colab).
video_path = "/content/WhatsApp Video 2025-02-22 at 13.40.45_bce8d8b3.mp4"  # Update with your file

# Initialize MediaPipe Holistic (includes face mesh).
mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic(
    static_image_mode=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)
mp_draw = mp.solutions.drawing_utils

# Create an ipywidgets Image widget for fast, in-place updates.
img_widget = widgets.Image(format='jpeg')
display(img_widget)

# ------------------ Utility Functions ------------------

def rotationMatrixToEulerAngles(R):
    """
    Convert a rotation matrix R to Euler angles (radians).
    Returns angles in the order [pitch, yaw, roll].
    """
    sy = math.sqrt(R[0, 0] ** 2 + R[1, 0] ** 2)
    singular = sy < 1e-6
    if not singular:
        x = math.atan2(R[2, 1], R[2, 2])
        y = math.atan2(-R[2, 0], sy)
        z = math.atan2(R[1, 0], R[0, 0])
    else:
        x = math.atan2(-R[1, 2], R[1, 1])
        y = math.atan2(-R[2, 0], sy)
        z = 0
    return np.array([x, y, z])

def draw_rounded_rectangle(img, top_left, bottom_right, color, thickness, radius=20):
    """
    Draw a rectangle with rounded corners on img.
    """
    x1, y1 = top_left
    x2, y2 = bottom_right

    # Draw straight lines
    cv2.line(img, (x1 + radius, y1), (x2 - radius, y1), color, thickness)
    cv2.line(img, (x1 + radius, y2), (x2 - radius, y2), color, thickness)
    cv2.line(img, (x1, y1 + radius), (x1, y2 - radius), color, thickness)
    cv2.line(img, (x2, y1 + radius), (x2, y2 - radius), color, thickness)

    # Draw arcs for corners
    cv2.ellipse(img, (x1 + radius, y1 + radius), (radius, radius), 180, 0, 90, color, thickness)
    cv2.ellipse(img, (x2 - radius, y1 + radius), (radius, radius), 270, 0, 90, color, thickness)
    cv2.ellipse(img, (x1 + radius, y2 - radius), (radius, radius), 90, 0, 90, color, thickness)
    cv2.ellipse(img, (x2 - radius, y2 - radius), (radius, radius), 0, 0, 90, color, thickness)

def draw_fancy_label(img, top_left, bottom_right, label):
    """
    Draw a fancy, semi-transparent rounded rectangle over the face
    and put the label ("Cheated" or "Normal") inside it.
    """
    x1, y1 = top_left
    x2, y2 = bottom_right

    # Create overlay for translucent background.
    overlay = img.copy()
    cv2.rectangle(overlay, (x1, y1), (x2, y2), (50, 50, 50), -1)
    alpha = 0.4  # transparency factor
    cv2.addWeighted(overlay, alpha, img, 1 - alpha, 0, img)

    # Draw a fancy rounded rectangle border.
    draw_rounded_rectangle(img, (x1, y1), (x2, y2), (0, 255, 255), thickness=3, radius=20)

    # Center the label text.
    font = cv2.FONT_HERSHEY_TRIPLEX
    font_scale = 1.2
    thickness_text = 2
    (text_w, text_h), _ = cv2.getTextSize(label, font, font_scale, thickness_text)
    text_x = x1 + (x2 - x1 - text_w) // 2
    text_y = y1 + (y2 - y1 + text_h) // 2

    # Add a shadow for fancy effect.
    cv2.putText(img, label, (text_x+2, text_y+2), font, font_scale, (0, 0, 0), thickness_text, cv2.LINE_AA)
    cv2.putText(img, label, (text_x, text_y), font, font_scale, (255, 255, 255), thickness_text, cv2.LINE_AA)

# ------------------ Main Loop ------------------

cap = cv2.VideoCapture(video_path)
yaw_threshold = 15  # degrees

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Finished processing video.")
        break

    # Process the frame.
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = holistic.process(frame_rgb)

    if results.face_landmarks:
        # Draw face mesh using face_mesh connections.
        mp_draw.draw_landmarks(frame, results.face_landmarks, mp.solutions.face_mesh.FACEMESH_TESSELATION)
        h, w, _ = frame.shape
        face_landmarks = results.face_landmarks.landmark

        # Use 6 key landmarks for head pose estimation.
        # Indices: Nose tip (1), Chin (152), Left eye left corner (33),
        # Right eye right corner (263), Left mouth corner (61), Right mouth corner (291)
        if len(face_landmarks) >= 468:
            image_points = np.array([
                (face_landmarks[1].x * w,   face_landmarks[1].y * h),    # Nose tip
                (face_landmarks[152].x * w, face_landmarks[152].y * h),    # Chin
                (face_landmarks[33].x * w,  face_landmarks[33].y * h),     # Left eye left corner
                (face_landmarks[263].x * w, face_landmarks[263].y * h),    # Right eye right corner
                (face_landmarks[61].x * w,  face_landmarks[61].y * h),     # Left mouth corner
                (face_landmarks[291].x * w, face_landmarks[291].y * h)     # Right mouth corner
            ], dtype=np.float64)

            model_points = np.array([
                [0.0,    0.0,    0.0],      # Nose tip
                [0.0,   -63.6,  -12.5],      # Chin
                [-43.3,  32.7,  -26.0],      # Left eye left corner
                [43.3,   32.7,  -26.0],      # Right eye right corner
                [-28.9, -28.9,  -24.1],      # Left mouth corner
                [28.9,  -28.9,  -24.1]       # Right mouth corner
            ], dtype=np.float64)

            focal_length = w
            center = (w / 2, h / 2)
            camera_matrix = np.array([
                [focal_length, 0, center[0]],
                [0, focal_length, center[1]],
                [0, 0, 1]
            ], dtype="double")
            dist_coeffs = np.zeros((4, 1))

            success, rotation_vector, translation_vector = cv2.solvePnP(
                model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_ITERATIVE
            )
            if success:
                rmat, _ = cv2.Rodrigues(rotation_vector)
                euler_angles = rotationMatrixToEulerAngles(rmat)
                euler_angles_deg = np.degrees(euler_angles)
                pitch, yaw, roll = euler_angles_deg

                # Determine label based on yaw.
                label = "Cheated" if abs(yaw) > yaw_threshold else "Normal"

                # Compute bounding box for the face using all face landmarks.
                xs = [lm.x * w for lm in face_landmarks]
                ys = [lm.y * h for lm in face_landmarks]
                x_min, x_max = int(min(xs)), int(max(xs))
                y_min, y_max = int(min(ys)), int(max(ys))

                # Add margin.
                margin = 20
                x1 = max(x_min - margin, 0)
                y1 = max(y_min - margin, 0)
                x2 = min(x_max + margin, w)
                y2 = min(y_max + margin, h)

                # Draw fancy rectangle with label.
                draw_fancy_label(frame, (x1, y1), (x2, y2), label)

    # Update display using ipywidgets for super-fast updates.
    ret2, buffer = cv2.imencode('.jpg', frame)
    if ret2:
        img_widget.value = buffer.tobytes()

cap.release()
print("Processing complete.")


ImportError: DLL load failed while importing _framework_bindings: A dynamic link library (DLL) initialization routine failed.